In [11]:
# 0 - Angry
# 1 - Disgust
# 2 - Fear
# 3 - Happy
# 4 - Sad
# 5 - Surprise
# 6 - Neutral

def get_model():
    from keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

    conv_kerr_size = (3, 3)
    maxpool_kerr_size = (2, 2)
    model = Sequential()
    model.add(Conv2D(32, conv_kerr_size, activation='relu', input_shape=(48, 48, 1)))
    model.add(Conv2D(32, conv_kerr_size, activation='relu', padding='same'))
    model.add(MaxPooling2D(maxpool_kerr_size))
    model.add(BatchNormalization())

    model.add(Conv2D(64, conv_kerr_size, activation='relu'))
    model.add(Conv2D(64, conv_kerr_size, activation='relu', padding='same'))
    model.add(MaxPooling2D(maxpool_kerr_size))
    model.add(BatchNormalization())

    model.add(Conv2D(128, conv_kerr_size, activation='relu'))
    model.add(Conv2D(128, conv_kerr_size, activation='relu', padding='same'))
    model.add(Conv2D(256, (5, 5), activation='relu'))
    model.add(Conv2D(256, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                      metrics=['acc'])
    return model
model = get_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 46, 46, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 23, 23, 32)        128       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 21, 21, 64)        18496     
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 21, 21, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 10, 10, 64)       

In [12]:
from keras.preprocessing.image import ImageDataGenerator
import os

img_size = (48, 48)
img_datagen = ImageDataGenerator(rescale=1/255)
img_batch_size = 128
train_generator = img_datagen.flow_from_directory('../data/standard/train',
                                                  target_size=img_size,
                                                  batch_size=img_batch_size,
                                                  color_mode='grayscale',
                                                  class_mode='categorical')
validation_generator = img_datagen.flow_from_directory('../data/standard/val/',
                                                   target_size=img_size,
                                                   batch_size=img_batch_size,
                                                   color_mode='grayscale',
                                                   class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [13]:
import keras.callbacks as clb
callbacks = [
    clb.ReduceLROnPlateau(monitor='val_acc', factor=0.1, min_lr=1e-7, patience=2, verbose=1),
    clb.EarlyStopping(monitor='val_acc', patience=4, verbose=1),
    # clb.ModelCheckpoint(monitor='val_acc', filepath='../models/standard_model.h5', save_best_only=True, verbose=1)
]
nr_of_train_pictures = 28709
train_steps = nr_of_train_pictures // img_batch_size + 1
nr_of_valid_pictures = 3589
valid_steps = nr_of_valid_pictures // img_batch_size + 1
history = model.fit(train_generator,
                    steps_per_epoch=train_steps,
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=valid_steps,
                    callbacks=callbacks
                    )

Epoch 1/100
225/225 [==============================] - 33s 144ms/step - loss: 0.2329 - acc: 0.1691 - val_loss: 0.2374 - val_acc: 0.1691

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b.', label='train acc')
plt.plot(epochs, val_acc, label='validation acc')
plt.legend()

plt.figure()

plt.plot(epochs, loss, '.', label='train loss')
plt.plot(epochs, val_loss, label='validation loss')
plt.legend()

plt.show()